In [77]:
import os 
import time
import requests  
import json  
import openai
import numpy as np
from openai import OpenAI
#from prompt_construction import read_prompt_list ,  prompt_construction module is not available here

api_key = os.environ["OPENAI_API_KEY"]  

%reload_ext autoreload
%autoreload 2

In [2]:
# Define the API endpoint URL  
def get_response_siemens_api(url, model, max_token, user_content, system_content):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': model,  
        'messages': [  
            {'role': 'user', 'content': user_content },
            {'role': 'system', 'content': system_content}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': max_token          
    }  
      
    # Make the POST request  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return print(response.json()["choices"][0]["message"]["content"])


In [3]:
url = 'https://api.siemens.com/llm/chat/completions'  
model = 'mistral-7b-instruct'
max_token = 100
user_content = ("Please classify the given requirement into 'functional requirement' or 'non-functional requirement'. "
                "The answer should be in format {the given requirement: functional requirement or non-functional requirement}."
                "The given requirement: The system shall refresh the display every 60 seconds." )
system_content = ("You are a senior software engineer who is experienced in software requirement classification! " )

get_response_siemens_api(url, model, max_token, user_content, system_content)

 {The system shall refresh the display every 60 seconds: functional requirement}


# define useful functions

In [ ]:
# Define the API endpoint URL  
def get_response_siemens_api(prompt):  
    # Define the request headers  
    headers = {  
        'accept': 'application/json',  
        'Authorization': 'Bearer {}'.format(api_key),  
        'Content-Type': 'application/json'  
    }  

    # Define the request payload  
    payload = {  
        'model': 'mistral-7b-instruct',  
        'messages': [  
            {'role': 'user', 'content': prompt },
            {'role': 'system', 'content': "You are a senior software engineer who is experienced in software requirement classification! "}           
        ],  
        'temperature': 0,  
        'stream': False,  
        'max_tokens': 6666          
    }  
      
    # Make the POST request  
    url = 'https://api.siemens.com/llm/chat/completions'  
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
      
    # Print the response  
    return print(response.json()["choices"][0]["message"]["content"])


In [82]:
def get_completion(prompt_list, save_path):
    """load the prompt_list, get the completion from llm, and save the results."""

    start_time = time.time()  
    
    completion_list = []
    for i, prompt in enumerate(prompt_list):
        print("The prompt: " + prompt + "\n######")
     

        while i < 10:
            try:   
                completion = get_response_siemens_api(prompt)
                print(completion)
                completion_list.append(completion)
                break
        
            except: #InvalidRequestError: in case that the prompt and output reach the token limitation
                print("Connection error: try again automatically")
            i = i+1    


        
        #completion = get_response_siemens_api(prompt)
        #print(completion)
        #completion_list.append(completion)
        #print(i, "th completion: ", completion)

    # calculate the execution time
    end_time = time.time()  
    print("execution time: {}".format(end_time - start_time), "second")

In [78]:
def read_prompt_list(path):
    """read the saved list of prompts 
    input
        path, string path to save the list
    """
    with open(path, 'r') as file:  
        prompt_list_read = []
        prompt = ""                            
        num_line_n = 0
        for line in file:
            # read the prompt one by one based on the double empty line /n/n/n
            if line == "\n":
                num_line_n = num_line_n + 1
                if num_line_n == 2:
                    prompt_list_read.append(prompt)
                    prompt = ""
                    num_line_n = 0
            else:
                prompt = prompt + "\n" + line  

    return prompt_list_read


In [83]:
#path = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0_bi.txt"
#path = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5_bi.txt"
path = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_0.txt"
#path = "/Users/yongjiantang/Desktop/tang/code/re_genai/processed_prompts/prompt_promise_random_5.txt"


prompt_list = read_prompt_list(path)
save_path = ""
get_completion(prompt_list, save_path)

The prompt: 
Please classify the given software requirements into functional requirement or non-functional requirement. The answer should be one word, i.e. Functional or Non-functional

Below are some demonstration examples for you to learn, which consist of a software requirement and its category: 

'The Disputes System must allow the user to create three unique types of ticket retrieval requests. The three types of ticket retrieval requests are (1) Request for original receipt (2) Request for a copy of the receipt or (3) Request for a portfolio. A portfolio consists of documentation that would provide proof of a purchase such as the documentation that is received from a car rental agency that is more than a sales receipt.': Functional

'The product shall generate a CMA report in an acceptable time.': Non-functional

'The Disputes application must conform to the legal requirements as specified by Regulation E and Regulation Z that govern credit card disputes processing.': Non-function

# get completion on promise dataset

## binary classification

## multi-class classification

# get completion on nfr dataset

## multi-class classification